# Resources:


1.   "Getting Started with Google BigQuery in Colab" Notebook by Google at this [link](https://colab.research.google.com/notebooks/bigquery.ipynb).
2.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [2]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [26]:
# Display query output immediately

%%bigquery --project testingbq-337212
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [40]:
# Save output in a variable `df`

%%bigquery --project testingbq-337212 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [41]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [29]:
# You might need to set up your personal project in Google BigQuery and use its ID below:
project_id = 'testingbq-337212'

### Sample approximately 2000 random rows

In [30]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [31]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,2107.000000,2107.000000,2107.000000,2107.000000,2107.000000,2107.000000,2107.000000,2016.000000,2016.000000,1623.000000,1623.000000,785.000000,785.000000,1883.000000,1883.000000,2069.000000,2069.000000,2034.000000,276.000000,2103.000000,1931.000000,106.000000
mean,511918.587091,91240.416706,1987.337447,6.595159,15.862364,52.229995,12.678215,41.865030,12.569444,1014.683857,11.078866,965.619617,11.927389,12.299310,12.280935,6.942581,12.619140,12.271632,25.584783,43.748264,0.070197,9.794340
std,299620.551401,25165.622099,15.738350,3.473251,8.850561,24.156092,7.866823,22.809407,7.849840,9.425294,7.383869,72.693735,7.801169,9.649318,7.769678,5.038566,7.833923,6.737631,8.980269,24.267347,0.262637,12.703789
min,10015.000000,13.000000,1930.000000,1.000000,1.000000,-50.400002,4.000000,-55.200001,4.000000,971.000000,4.000000,607.099976,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,6.000000,-58.400002,0.000000,0.400000
25%,260380.000000,99999.000000,1977.000000,4.000000,8.000000,39.049999,6.000000,29.299999,6.000000,1009.299988,6.000000,951.599976,6.000000,6.500000,6.000000,3.400000,6.000000,7.800000,19.000000,31.799999,0.000000,2.000000
50%,543770.000000,99999.000000,1989.000000,7.000000,16.000000,55.000000,8.000000,44.599998,8.000000,1014.500000,8.000000,994.200012,8.000000,9.900000,8.000000,5.900000,8.000000,11.100000,23.900000,46.400002,0.000000,6.100000
75%,725111.500000,99999.000000,2000.000000,10.000000,23.000000,71.199997,23.000000,57.299999,23.000000,1020.000000,18.000000,1007.799988,23.000000,14.900000,23.000000,9.300000,23.000000,15.500000,31.100000,61.200001,0.010000,11.000000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,99.500000,24.000000,79.800003,24.000000,1057.000000,24.000000,1032.300049,24.000000,99.400002,24.000000,39.799999,24.000000,55.900002,55.900002,89.599998,4.090000,93.699997


### View the first 10 rows

In [32]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,577070,99999,1982,12,15,33.599998,7,29.299999,7.0,NaN,NaN,856.000000,7.0,4.4,7.0,1.4,7.0,3.9,NaN,31.299999,True,None,None,0.01,NaN,False,False,False,False,False,False
1,296260,99999,1949,9,30,48.200001,4,NaN,NaN,1011.000000,4.0,NaN,NaN,NaN,NaN,5.7,4.0,8.9,NaN,37.000000,False,None,None,0.00,NaN,False,False,False,False,False,False
2,588200,99999,1987,2,25,52.700001,4,46.599998,4.0,1022.400024,4.0,NaN,NaN,8.9,4.0,1.0,4.0,1.9,NaN,41.900002,False,None,None,0.00,NaN,True,True,True,True,True,True
3,969350,99999,1985,11,8,85.400002,7,73.599998,7.0,1009.500000,6.0,NaN,NaN,6.5,7.0,5.7,7.0,12.0,NaN,75.199997,True,None,None,0.00,NaN,False,False,False,False,False,False
4,24110,99999,2002,6,11,58.099998,6,55.400002,6.0,1008.599976,6.0,1000.700012,6.0,18.4,6.0,4.5,6.0,13.6,NaN,52.500000,False,None,None,0.11,NaN,False,False,False,False,False,False
5,221650,99999,1960,7,10,56.599998,7,41.299999,7.0,1015.900024,7.0,NaN,NaN,12.4,7.0,15.0,7.0,19.0,NaN,46.000000,False,None,None,0.00,NaN,False,False,False,False,False,False
6,539750,99999,1973,9,14,67.699997,7,59.500000,7.0,1015.799988,7.0,NaN,NaN,6.2,7.0,2.2,7.0,7.8,NaN,60.799999,False,None,None,0.08,NaN,False,False,False,False,False,False
7,77850,99999,1995,4,17,57.099998,7,48.500000,7.0,1009.799988,7.0,996.700012,7.0,12.8,7.0,10.8,7.0,15.9,NaN,48.900002,False,None,None,0.00,NaN,False,False,False,False,False,False
8,154900,99999,1998,8,7,77.400002,11,65.500000,7.0,1022.799988,7.0,1018.400024,7.0,6.2,11.0,2.6,11.0,7.8,NaN,71.599998,False,None,None,0.00,NaN,False,False,False,False,False,False
9,269970,99999,1977,7,22,68.699997,8,59.200001,8.0,1014.200012,8.0,NaN,NaN,10.9,8.0,3.2,8.0,7.8,NaN,59.000000,False,None,None,0.00,NaN,False,False,False,False,False,False


In [33]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
847,427240,1984,9,2,4.09
560,228870,1965,11,13,2.95
1229,355420,1960,10,30,2.95
1419,470500,1990,8,10,2.64
651,422600,1990,7,18,2.64
1767,489200,2003,7,19,2.56
2055,783880,1975,11,17,2.48
212,722560,2007,6,18,2.40
1218,474350,1998,8,28,2.20
1666,700001,2009,9,17,2.16


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [34]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [35]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316


# Example: Access to Public Dataset "Hacker News"

In [55]:
# Let's reference the dataset:
dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [56]:
# Listing tables:
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

comments
full
full_201510
stories


In [57]:
# Checking the table schema:
table_ref = dataset_ref.table('comments')
table = client.get_table(table_ref)

In [58]:
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [59]:
# Show data in a dataframe:
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


Hacker News sends awards to those, who wrote more than 10000 posts. Let's find authors with more than 10000 posts (we will call the column with post counts as "NumPosts"):

In [62]:
prolific_commenters_query = '''
                            SELECT author, COUNT(1) as NumPosts
                            FROM `bigquery-public-data.hacker_news.comments`
                            GROUP BY author
                            HAVING COUNT(1)>10000
                            '''

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

prolific_commenters = query_job.to_dataframe()
print(prolific_commenters.head())


    author  NumPosts
0    DanBC     12902
1    sp332     10882
2   davidw     10764
3  rayiner     11080
4  tptacek     33839


# Example: Access to Public Dataset "World Bank International Education"

In [50]:
dataset_ref = client.dataset('world_bank_intl_education', project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [51]:
tables = list (client.list_tables(dataset))

for table in tables:
  print(table.table_id)

country_series_definitions
country_summary
international_education
series_summary


In [52]:
table_ref = dataset_ref.table('international_education')
table = client.get_table(table_ref)

table.schema

[SchemaField('country_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('country_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_name', 'STRING', 'NULLABLE', '', ()),
 SchemaField('indicator_code', 'STRING', 'NULLABLE', '', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', '', ()),
 SchemaField('year', 'INTEGER', 'NULLABLE', '', ())]

In [53]:
# show data as a dataframe
client.list_rows(table, max_results=10).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,St. Vincent and the Grenadines,VCT,Official entrance age to post-secondary non-te...,UIS.THAGE.4.A.GPV,1.700000e+01,2016
1,Suriname,SUR,GNI (current US$),NY.GNP.MKTP.CD,3.167325e+09,2016
2,Suriname,SUR,SABER: (Education Management Information Syste...,SABER.EMIS.GOAL2.LVL3,3.000000e+00,2016
3,Tajikistan,TJK,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,8.458801e+09,2016
4,Tajikistan,TJK,Official entrance age to pre-primary education...,UIS.THAGE.0,3.000000e+00,2016
5,Tajikistan,TJK,Population of the official entrance age to pri...,UIS.SAP.1.G1,1.871570e+05,2016
6,Marshall Islands,MHL,Population of the official age for the last gr...,SP.PRM.GRAD.TO,1.445000e+03,2016
7,Mauritania,MRT,Percentage of graduates from tertiary educatio...,SE.TER.GRAD.EN.ZS,8.632620e+00,2016
8,"Micronesia, Fed. Sts.",FSM,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,2.962826e+08,2016
9,"Micronesia, Fed. Sts.",FSM,GDP per capita (constant 2005 US$),NY.GDP.PCAP.KD,2.823434e+03,2016


Which countries spent the largest fraction of GDP on education for the years 2010 - 2017?

Let's use indicator_code SE:XPD.TOTL.GD.ZS, which shows 'Government expenditure on education a % of GDP'.

In [54]:
country_spend_pct_query = '''
                            SELECT country_name, AVG(value) AS avg_ed_spending_pct
                            FROM `bigquery-public-data.world_bank_intl_education.international_education`
                            WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND
                            year BETWEEN 2010 AND 2017
                            GROUP BY country_name
                            ORDER BY AVG(value) DESC
                            '''
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)
country_spending_results = country_spend_pct_query_job.to_dataframe()
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


# Example: Access to Public Dataset "Openaq"

In [63]:
dataset_ref = client.dataset('openaq', project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

In [64]:
tables = list (client.list_tables(dataset))

for table in tables:
  print(table.table_id)

global_air_quality


In [65]:
table_ref = dataset_ref.table('global_air_quality')
table = client.get_table(table_ref)

table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', ()),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', ()),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', ()),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', ()),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', ()),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', ()),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', ()),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', ()),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', 'Longitude in d

In [87]:
# show data as a dataframe
client.list_rows(table, max_results=10).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
5,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,co,840.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
6,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,no2,166.55,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
7,BWSSB Kadabesanahalli,Bengaluru,IN,o3,17.11,2017-02-12 01:45:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
8,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,pm25,40.94,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25
9,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,so2,6.63,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25


Which countries reported pollution in units "ppm"?

In [71]:
country_pollution_query= '''
                            SELECT country,
                            FROM `bigquery-public-data.openaq.global_air_quality`
                            WHERE unit = 'ppm' 
                            '''
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
country_pollution_query_job = client.query(country_pollution_query, job_config=safe_config)
country_pollution_results = country_pollution_query_job.to_dataframe()
print(country_pollution_results.head())

  country
0      US
1      US
2      US
3      US
4      US


Which pollution levels were reported as 0?

In [77]:
zero_pollution_query = '''
                          SELECT location, country, city, value, unit
                          FROM `bigquery-public-data.openaq.global_air_quality`
                          WHERE value=0
                          LIMIT 10
                          '''
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
zero_pollution_query_job = client.query(zero_pollution_query, job_config=safe_config)
zero_pollution_results = zero_pollution_query_job.to_dataframe()
print(zero_pollution_results.head())

                                        location country     city  value   unit
0                     Victoria Memorial - WBSPCB      IN  Kolkata    0.0  µg/m³
1  Rabindra Bharati University, Kolkata - WBSPCB      IN  Kolkata    0.0  µg/m³
2                   Zamość ul. Hrubieszowska 69A      PL   Zamość    0.0  µg/m³
3                               Końskie, MOBILNA      PL  Końskie    0.0  µg/m³
4                               Końskie, MOBILNA      PL  Końskie    0.0  µg/m³


What are the cities with the highest pollution values?

In [122]:
max_pollution_query = '''
                          SELECT 
                            city, unit, MAX(value) AS pollution_value
                          FROM `bigquery-public-data.openaq.global_air_quality`
                          GROUP BY city, unit
                          ORDER BY pollution_value DESC
                          LIMIT 5
                          '''
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
max_pollution_query_job = client.query(max_pollution_query, job_config=safe_config)
max_pollution_results = max_pollution_query_job.to_dataframe()
print(max_pollution_results.head())

           city   unit  pollution_value
0  Escuela E-10  µg/m³         825736.0
1         Satna  µg/m³          80000.0
2        Toledo  µg/m³          50000.0
3         Delhi  µg/m³          50000.0
4       Granada  µg/m³          35122.0


In [123]:
# filter aggregated pollution_value with HAVING clause
max_pollution_query_2 = '''
                          SELECT 
                            country, city, unit, MAX(value) AS pollution_value
                          FROM `bigquery-public-data.openaq.global_air_quality`
                          GROUP BY country, city, unit
                          HAVING pollution_value > 50000
                          ORDER BY pollution_value DESC
                          LIMIT 5
                          '''
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
max_pollution_query_job = client.query(max_pollution_query_2, job_config=safe_config)
max_pollution_results = max_pollution_query_job.to_dataframe()
print(max_pollution_results.head())

  country          city   unit  pollution_value
0      CL  Escuela E-10  µg/m³         825736.0
1      IN         Satna  µg/m³          80000.0
